<img style="float: right;" src="https://drive.google.com/uc?export=view&id=1qVA-sPHp5TY5D7enbU7mF4zZ8obu5wrC" width=150 height=150 />

# CWatM: Community Water Model
### CWatM is an open-source hydrological model developed at IIASA
This notebook runs CWatM using the excel file cwatm_settings.xlsx.
<br><br>

The **output_folder** in the accompanied notebook should point to where the simulations are held. <br>
The **basepath** should point to the folder "CWatM_settings" containing:
 - *cwatm_settings.xlsx*: 
 - *settings_CwatM_template.ini*

In [1]:
basepath = 'C:\GitHub\FUSE\CWatM_settings'
#For nira_repo change in 3 locations
#basepath = r'C:\GitHub\nira_repo\modules\hydro\hydro_files'
water_balance = False
outputsForNotebooks = True
AgricultureNotebook = True
AgInputsNotebook = True
print_settings = False

In [2]:
def create_cwatm_settings_dict_obj_TEST(xl_settings_file_path):
    import xlrd
    import pandas as pd
    # Create dictionary from excel file
    settings_dict = {}
    df = pd.read_excel(xl_settings_file_path)
    wb = xlrd.open_workbook(xl_settings_file_path)
    sh = wb.sheet_by_index(0)
    for i in range(len(df)+1): #MS
        cell_key = sh.cell(i, 0).value
        cell_value = sh.cell(i, 1).value
        settings_dict[cell_key] = cell_value
    return settings_dict
    
def define_cwatm_settings_TEST(settings_template, new_settings_file, settings_dictionary):
    global Modflow
    sourcefile = open(settings_template, "r")
    newfile = open(new_settings_file, "wt")
    keylist = list(settings_dictionary.keys())
    for key in keylist:
        if 'modflow_coupling' in key:
            Modflow = settings_dictionary[key]
            Modflow_found = True
        
    for line in sourcefile:

        boolcheck = [elem in line for elem in keylist]
        match = any(elem in line for elem in keylist)
        ix = boolcheck.index(match)
        if 'modflow_coupling' in line and not 'Modflow_found' in locals():
            Modflow = False if line.split()[-1]==('False' or 'FALSE') else True
        if match:
            key = keylist.pop(ix)
            value = settings_dictionary[key]
            
            water_balance_outputs=''
            if key == 'OUT_MAP_Daily':
                if water_balance:
                    #water_balance_outputs= 'pumping_actual, gwstore, returnFlow, act_SurfaceWaterAbstract,act_nonIrrConsumption,sum_runoff,sum_interceptEvap,sum_actBareSoilEvap,act_totalIrrConsumption, sum_gwRecharge, gwstore, capillar, baseflow, act_bigLakeResAbst, leakage, actTransTotal_nonpaddy, actTransTotal_paddy, actTransTotal_grasslands, actTransTotal_forest, pumping_actual, pumping, cellArea, totalET, Rain, EvapoChannel, EvapWaterBodyM,  channelStorage, lakeResStorage, totalSto, storGroundwater, gwstore, sum_actTransTotal, sum_actBareSoilEvap, sum_interceptEvap, sum_openWaterEvap, addtoevapotrans,   lakeResInflowDis, lakeResOutflowDis, lakeResOutflowM, lakeResInflowM, discharge'
                    water_balance_outputs= 'lakeResInflowDis, lakeResOutflowDis, head_development, act_channelAbstract_Lift, act_nonLocalchannelAbstract_M, act_nonLocalLakeAbstract_M, sum_prefFlow_GW, sum_perc3toGW_GW, leakageIntoGw, leakageIntoRunoff,  riverbedExchangeM, act_bigLakeResAbst_alloc, act_LocalLakeAbstract, act_channelAbstract_Local, act_channelAbstract, act_irrWithdrawalGW, act_irrWithdrawalSW, act_nonIrrWithdrawalGW, act_nonIrrWithdrawalSW, act_nonIrrWithdrawal, act_irrWithdrawal, delivered_water, act_LocalLakeAbstract, resStorageTotal_alloc, pumping_actual, gwstore, returnFlow, act_SurfaceWaterAbstract,act_nonIrrConsumption,sum_runoff,sum_interceptEvap,sum_actBareSoilEvap,act_totalIrrConsumption, sum_gwRecharge, gwstore, capillar, baseflow, act_bigLakeResAbst, act_bigLakeResAbst_alloc, leakage, actTransTotal_nonpaddy, actTransTotal_paddy, actTransTotal_grasslands, actTransTotal_forest, pumping_actual, pumping, cellArea, totalET, Rain, EvapoChannel, EvapWaterBodyM,  channelStorage, lakeResStorage, totalSto, storGroundwater, gwstore, sum_actTransTotal, sum_actBareSoilEvap, sum_interceptEvap, sum_openWaterEvap, addtoevapotrans, lakeResOutflowM, lakeResInflowM, discharge'

                elif outputsForNotebooks:
                    water_balance_outputs= 'cropKC[1], discharge, lakeResInflowDis, lakeResOutflowDis, lakeResStorage, act_bigLakeResAbst_alloc, act_LocalLakeAbstract, resStorageTotal_alloc'
                else:
                    water_balance_outputs='discharge'
                if str(value)!='':
                    water_balance_outputs += ','

            newstring = key + ' = ' + water_balance_outputs + str(value) + '\n'
            newfile.write(newstring)
            
        else:
            newfile.write(line)
    sourcefile.close()
    newfile.close()

def show_cwatm_settings_file_TEST(settings_file_path):
    file = open(settings_file_path, "r")
    for line in file:
        print(line)
    file.close()

def run_cwatm_TEST(settings_file_path):
    import  os
    cwatm_source_path = 'C:\GitHub\CWATM_priv\source_py3\cwatm3.py'
    #cwatm_source_path = r'C:\GitHub\nira_repo\python_packages\source_py3\cwatm3.py'
    command_str = 'python ' + cwatm_source_path + " " + settings_file_path + ' -l'
    flag = os.system(command_str)
    if flag == 0:
        print('Successfully Finished Running CWATM\n')

In [ ]:
import time
from datetime import date

today = date.today().strftime("%B%d")

settings_template = basepath + r'\settings_CWatM_template.ini'
excel_settings_file =basepath + r'\cwatm_settings.xlsx'
#excel_settings_file =basepath + r'\cwatm_settings_12Oct.xlsx'
new_settings_file = basepath + r'\settings_CWatM_'+today+'.ini'

# Create an initial settings dictionary object from the excel file.
# This dictionary be modified by the urban and ag calculations to pass file paths or values
settings_dict = create_cwatm_settings_dict_obj_TEST(excel_settings_file)

define_cwatm_settings_TEST(settings_template, new_settings_file, settings_dict)

if print_settings == True:
    show_cwatm_settings_file_TEST(new_settings_file)
    

run_cwatm_TEST(new_settings_file)

In [ ]:
run ./1_Balance_CWatM.ipynb